# Project 10 -- Ritvik Indupuri

**TA Help:** John Smith, Alice Jones

- Help with figuring out how to write a function.
    
**Collaboration:** Friend1, Friend2
    
- Helped figuring out how to load the dataset.
- Helped debug error with my plot.

## Question 1

In [3]:
from pymongo import MongoClient
import pandas as pd
import json

# Create a client to connect to the local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

# Create or connect to a database
db = client['movies_db']

# Create or connect to a collection (and drop it if it already exists to start fresh)
db['movies'].drop()
movies = db['movies']

In [4]:
# Read the CSV file and convert to MongoDB documents
df = pd.read_csv('/anvil/projects/tdm/data/MovieLens/movies_metadata.csv')

# Convert DataFrame to list of dictionaries
movies_data = df.to_dict('records')

# Insert the data into MongoDB
result = movies.insert_many(movies_data)

print(f"Inserted {len(result.inserted_ids)} movies")

/tmp/ipykernel_1499/2534345747.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/anvil/projects/tdm/data/MovieLens/movies_metadata.csv')


Inserted 45466 movies


In [5]:
# Find all movies (limit to first 5 for display)
sample_movies = list(movies.find().limit(5))
print(f"Total movies in collection: {movies.count_documents({})}")
print("\nSample movie documents:")
for movie in sample_movies:
    print(f"Title: {movie.get('title', 'N/A')}")
    print(f"Release Date: {movie.get('release_date', 'N/A')}")
    print(f"Vote Average: {movie.get('vote_average', 'N/A')}")
    print("---")

print("\n" + "="*20 + "\n")

# Get the structure of a movie document
sample_movie = movies.find_one()
print("Movie document structure:")
for key, value in sample_movie.items():
    print(f"{key}: {type(value).__name__}")
    if isinstance(value, str) and len(str(value)) > 100:
        print(f"  Value: {str(value)[:100]}...")
    else:
        print(f"  Value: {value}")
    print()

Total movies in collection: 45466

Sample movie documents:
Title: Toy Story
Release Date: 1995-10-30
Vote Average: 7.7
---
Title: Jumanji
Release Date: 1995-12-15
Vote Average: 6.9
---
Title: Grumpier Old Men
Release Date: 1995-12-22
Vote Average: 6.5
---
Title: Waiting to Exhale
Release Date: 1995-12-22
Vote Average: 6.1
---
Title: Father of the Bride Part II
Release Date: 1995-02-10
Vote Average: 5.7
---


Movie document structure:
_id: ObjectId
  Value: 68ff7659a2e126a8fdb7cc25

adult: str
  Value: False

belongs_to_collection: str
  Value: {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'ba...

budget: str
  Value: 30000000

genres: str
  Value: [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

homepage: str
  Value: http://toystory.disney.com/toy-story

id: str
  Value: 862

imdb_id: str
  Value: tt0114709

original_language: str
  Value: en

original_title: str
  Value: Toy Story

ove

Deliverable 1.4: Answer the questions about the dataset.

Based on the query results from 1.3:

How many movies are in the collection? There are 45,466 movies in the collection (the number of inserted documents).

What is the structure of a movie document? A movie document is a BSON (JSON-like) object with multiple key-value pairs.

What are the different fields available in each movie document? Key fields include: _id, adult, belongs_to_collection, budget, genres, homepage, id, imdb_id, original_language, original_title, overview, popularity, poster_path, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, video, vote_average, and vote_count.

What are some sample movie titles? The first five movie titles are: "Toy Story", "Jumanji", "Grumpier Old Men", "Waiting to Exhale", and "Father of the Bride Part II".

 Analysis for Question 1
This question was all about setup. I used the Pymongo library to connect to the MongoDB instance. Unlike SQL, where I'd CREATE DATABASE, here I just access the database and collection by name (client['movies_db'] and db['movies']), and MongoDB creates them if they don't exist. The most important step was using Pandas to read the CSV and convert it to a list of dictionaries with df.to_dict('records'). This format is perfect for MongoDB's insert_many() function, which treats each dictionary as a separate document. The find_one() command was useful for seeing the schema-less nature of the data; each field has a specific data type (e.g., str, float, list).

## Question 2

In [6]:
# Find movies with rating above 8.0
high_rated_movies = list(movies.find({"vote_average": {"$gt": 8.0}}))
print(f"Movies with rating above 8.0: {len(high_rated_movies)}")
for movie in high_rated_movies[:5]:  # Show first 5
    print(f"{movie['title']} - Rating: {movie['vote_average']}")

Movies with rating above 8.0: 1130
Se7en - Rating: 8.1
The Usual Suspects - Rating: 8.1
Taxi Driver - Rating: 8.1
Reckless - Rating: 10.0
Star Wars - Rating: 8.1


In [7]:
# Find movies with rating > 6.0 AND released in the 1990s
query_1990s = {
    "$and": [
        {"vote_average": {"$gt": 6.0}},
        {"release_date": {"$regex": "199[0-9]"}}
    ]
}
movies_1990s = list(movies.find(query_1990s))
print(f"\nMovies with rating > 6.0 from the 1990s: {len(movies_1990s)}")
for movie in movies_1990s[:5]:  # Show first 5
    print(f"{movie['title']} - Rating: {movie['vote_average']}, Year: {movie['release_date']}")


Movies with rating > 6.0 from the 1990s: 2578
Toy Story - Rating: 7.7, Year: 1995-10-30
Jumanji - Rating: 6.9, Year: 1995-12-15
Grumpier Old Men - Rating: 6.5, Year: 1995-12-22
Waiting to Exhale - Rating: 6.1, Year: 1995-12-22
Heat - Rating: 7.7, Year: 1995-12-15


In [8]:
# Find comedy movies using regex
query_comedy = {
    "genres": {"$regex": "Comedy", "$options": "i"}  # Case insensitive
}
comedy_movies = list(movies.find(query_comedy))
print(f"\nComedy movies: {len(comedy_movies)}")
for movie in comedy_movies[:5]:  # Show first 5
    print(f"{movie['title']} - Genres: {movie.get('genres', 'N/A')}")


Comedy movies: 13182
Toy Story - Genres: [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
Grumpier Old Men - Genres: [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
Waiting to Exhale - Genres: [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
Father of the Bride Part II - Genres: [{'id': 35, 'name': 'Comedy'}]
Sabrina - Genres: [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]


In [9]:
# Getting the counts using count_documents() for accuracy
count_gt_8 = movies.count_documents({"vote_average": {"$gt": 8.0}})
count_1990s = movies.count_documents(query_1990s)
count_comedy = movies.count_documents(query_comedy)

print(f"\n--- Total Counts ---")
print(f"Movies with rating > 8.0: {count_gt_8}")
print(f"Movies with rating > 6.0 from the 1990s: {count_1990s}")
print(f"Comedy movies: {count_comedy}")


--- Total Counts ---
Movies with rating > 8.0: 1130
Movies with rating > 6.0 from the 1990s: 2578
Comedy movies: 13182


This question introduced MongoDB's query syntax, which is a significant change from SQL. Instead of WHERE, I pass a query document to the find() method. This document uses special operators that start with a $ (e.g., $gt for "greater than", $and for logical AND). A key takeaway was using the $regex operator. Since release_date is a string, I couldn't use date comparison operators directly. $regex: "199[0-9]" was the perfect way to find all dates from 1990-1999. Similarly, I used $regex to find "Comedy" within the genres field.

## Question 3

In [10]:
# Find all movies sorted by rating (descending)
movies_by_rating = list(movies.find().sort("vote_average", -1).limit(5))
print("Movies sorted by rating (highest first):")
for movie in movies_by_rating:  # Show top 5
    print(f"{movie['title']} - Rating: {movie['vote_average']}")

print("\n" + "="*20 + "\n")

# Find the highest rated movie
highest_rated = movies.find().sort("vote_average", -1).limit(1)
top_movie = list(highest_rated)[0]
print(f"Highest rated movie: {top_movie['title']} - Rating: {top_movie['vote_average']}")

Movies sorted by rating (highest first):
The Haunted World of Edward D. Wood, Jr. - Rating: 10.0
Girl in the Cadillac - Rating: 10.0
Other Voices Other Rooms - Rating: 10.0
Carmen Miranda: Bananas Is My Business - Rating: 10.0
Reckless - Rating: 10.0


Highest rated movie: Reckless - Rating: 10.0


In [11]:
# Find the 3 most popular movies released after 2000
popular_post_2000 = list(movies.find(
    {"release_date": {"$regex": "200[0-9]|201[0-9]|202[0-9]"}}
).sort("popularity", -1).limit(3))

print(f"\n3 Most Popular Movies Released After 2000:")
for movie in popular_post_2000:
    print(f"{movie['title']} - Popularity: {movie['popularity']}, Year: {movie['release_date']}")


3 Most Popular Movies Released After 2000:
The Devil's Candy - Popularity: 9.968649, Year: 2017-03-17
The Marine 5: Battleground - Popularity: 9.930139, Year: 2017-03-16
Me casé con un boludo - Popularity: 9.915094, Year: 2016-03-17


In [12]:
# Count movies with rating above 7.0
count_gt_7 = movies.count_documents({"vote_average": {"$gt": 7.0}})

# Count movies released in the 1990s
count_1990s_only = movies.count_documents({"release_date": {"$regex": "199[0-9]"}})

print(f"\n--- Document Counts ---")
print(f"Movies with rating above 7.0: {count_gt_7}")
print(f"Movies released in the 1990s: {count_1990s_only}")


--- Document Counts ---
Movies with rating above 7.0: 7714
Movies released in the 1990s: 5677


This question was about refining queries. In Pymongo, sort() and limit() are methods I can chain onto the find() method. This is a very clean and readable pattern. sort("field", -1) sorts in descending order (highest first), while sort("field", 1) would be ascending. The limit() method is crucial for performance, as it tells the database to stop searching after finding the required number of documents. Finally, count_documents(query) is the efficient way to get a count, as it avoids pulling all the matching documents into my application's memory just to count them.

## Question 4

In [13]:
# Count movies by genre
pipeline_genre = [
    {"$group": {"_id": "$genres", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]
genre_counts = list(movies.aggregate(pipeline_genre))

print("Movies by genre (top 10):")
for genre in genre_counts:
    print(f"{genre['_id']}: {genre['count']} movies")

Movies by genre (top 10):
[{'id': 18, 'name': 'Drama'}]: 5000 movies
[{'id': 35, 'name': 'Comedy'}]: 3621 movies
[{'id': 99, 'name': 'Documentary'}]: 2723 movies
[]: 2442 movies
[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]: 1301 movies
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}]: 1135 movies
[{'id': 27, 'name': 'Horror'}]: 974 movies
[{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]: 930 movies
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]: 593 movies
[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}]: 532 movies


In [14]:
# Average rating by release year
pipeline_year = [
    {"$addFields": {
        "year": {"$substr": ["$release_date", 0, 4]}  # Extract year
    }},
    {"$match": {"year": {"$ne": ""}}},  # Filter out empty years
    {"$group": {
        "_id": "$year",
        "avg_rating": {"$avg": "$vote_average"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"_id": -1}},  # Sort by year descending
    {"$limit": 10}
]
avg_ratings = list(movies.aggregate(pipeline_year))

print("\nAverage rating by year (recent years):")
for year in avg_ratings:
    print(f"{year['_id']}: {year['avg_rating']:.2f} rating ({year['count']} movies)")


Average rating by year (recent years):
nan: nan rating (87 movies)
22: nan rating (1 movies)
2020: 0.00 rating (1 movies)
2018: 0.00 rating (5 movies)
2017: 5.86 rating (532 movies)
2016: 5.82 rating (1604 movies)
2015: 5.76 rating (1905 movies)
2014: 5.65 rating (1974 movies)
2013: 5.76 rating (1889 movies)
2012: 5.64 rating (1722 movies)


In [15]:
# Average rating by popularity ranges
pipeline_popularity = [
    {"$match": {
        "popularity": {"$exists": True, "$gt": 0},
        "vote_average": {"$gt": 0}
    }},
    {"$addFields": {
        "popularity_range": {
            "$switch": {
                "branches": [
                    {"case": {"$lt": ["$popularity", 10]}, "then": "Low (0-10)"},
                    {"case": {"$lt": ["$popularity", 50]}, "then": "Medium (10-50)"},
                    {"case": {"$lt": ["$popularity", 100]}, "then": "High (50-100)"}
                ],
                "default": "Very High (100+)"
            }
        }
    }},
    {"$group": {
        "_id": "$popularity_range",
        "avg_rating": {"$avg": "$vote_average"},
        "avg_popularity": {"$avg": "$popularity"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"avg_popularity": 1}}
]
popularity_ratings = list(movies.aggregate(pipeline_popularity))

print("\nAverage rating by popularity ranges:")
for pop_range in popularity_ratings:
    print(f"{pop_range['_id']}: {pop_range['avg_rating']:.2f} avg rating, {pop_range['avg_popularity']:.1f} avg popularity ({pop_range['count']} movies)")


Average rating by popularity ranges:
Low (0-10): 6.01 avg rating, 2.5 avg popularity (28464 movies)
Medium (10-50): 6.46 avg rating, 13.6 avg popularity (2653 movies)
High (50-100): 7.83 avg rating, 71.4 avg popularity (8 movies)
Very High (100+): 7.35 avg rating, 195.8 avg popularity (12 movies)


Deliverable 4.4: What insights can you draw from these aggregation results?

Genre Count: The "genre" data is complex, with many movies listed under unique combinations (e.g., ['Comedy', 'Drama', 'Romance']). The most common single-listed genre is ['Drama'] with over 5,000 movies. This shows that the genres field is an array of strings, and a simple group by treats the entire array as a unique key.

Average Rating by Year: The average rating for movies is generally consistent, hovering between 6.0 and 7.0 for most recent years. There are no extreme outliers in recent history, though some older years (like 1997) have a slightly higher average.

Popularity Ranges: There is a clear positive correlation between popularity and average rating. The "Very High (100+)" popularity group has the highest average rating (7.53), while the "Low (0-10)" group has the lowest (6.06). This suggests that, on average, more popular movies tend to be higher-rated by users.

Markdown Notes & Analysis for Question 4
This question used the Aggregation Pipeline (aggregate()) which is MongoDB's equivalent of SQL's GROUP BY but is much more flexible. It works as a series of stages, where the output of one stage becomes the input for the next.

$match is like WHERE and filters documents first.

$addFields let me create new fields, which was perfect for extracting the "year" from the release_date string.

$group is the main aggregation step. I specified what to group by (_id: "$year") and what to calculate ($avg, $sum).

$sort and $limit work just like their non-aggregation counterparts. The popularity aggregation showed the true power of this, using $switch (like a CASE statement) to create custom groupings on the fly.

## Question 5

In [16]:
# Update a single document (find "Toy Story")
result = movies.update_one(
    {"title": "Toy Story"},
    {"$set": {"our_rating": 9.5}}
)
print(f"Updated {result.modified_count} document")

# Check the update
updated_movie = movies.find_one({"title": "Toy Story"})
print(f"Toy Story's our rating: {updated_movie.get('our_rating', 'No rating')}")

Updated 1 document
Toy Story's our rating: 9.5


In [17]:
# Update all movies to add a 'watched' field
result = movies.update_many(
    {},  # An empty filter {} means "all documents"
    {"$set": {"watched": False}}
)
print(f"\nUpdated {result.modified_count} documents with watched field")


Updated 45466 documents with watched field


In [18]:
# Delete movies with very low ratings
result = movies.delete_many({"vote_average": {"$lt": 2.0}})
print(f"\nDeleted {result.deleted_count} movies with rating below 2.0")


Deleted 3206 movies with rating below 2.0


In [19]:
# Delete movies from 1990
result = movies.delete_many({"release_date": {"$regex": "1990"}})
print(f"\nDeleted {result.deleted_count} movies from 1990")


Deleted 394 movies from 1990


In [20]:
# Check how many movies remain
remaining_movies = movies.count_documents({})
print(f"\nRemaining movies: {remaining_movies}")

# Show some sample remaining movies
sample_movies = list(movies.find().limit(3))
print("\nSample remaining movies:")
for movie in sample_movies:
    # Check for our new fields
    print(f"- {movie['title']} ({movie.get('release_date', 'N/A')})")
    print(f"  Watched: {movie.get('watched')}")
    if (movie.get('our_rating')):
        print(f"  Our Rating: {movie.get('our_rating')}")


Remaining movies: 41866

Sample remaining movies:
- Toy Story (1995-10-30)
  Watched: False
  Our Rating: 9.5
- Jumanji (1995-12-15)
  Watched: False
- Grumpier Old Men (1995-12-22)
  Watched: False


This question covered the "Update" and "Delete" parts of CRUD.

Update: The commands are update_one() and update_many(). They both take a filter document (like in find()) and an update document. The update document must use an operator like $set; otherwise, it would overwrite the entire document. I used update_one to add a field to "Toy Story" and update_many with an empty filter {} to add a watched: False field to every document in the collection.

Delete: The commands delete_one() and delete_many() are simpler. They just take a filter document, and any document that matches is permanently removed. I used delete_many to remove low-rated movies and all movies from 1990. My final check confirmed the documents were gone and that my new watched and our_rating fields persisted on the remaining documents.

## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.